In [ ]:
import urllib.request
import os
import re
import random
import string

def url_open(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent','Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:36.0) Gecko/20100101 Firefox/36.0')
    response = urllib.request.urlopen(req,timeout=90)
    return response.read()

# 获取page页码 1........9
def get_page_id_arr(url):
    html = url_open(url).decode('utf-8')
#     print(html)
    pattern = r'href="([^"]*)"[^>]*>.+?' # 匹配href
    page_id_arr = re.findall(pattern, html)
    page_id_arr_enable = []  # 过滤不需要的链接
    for id in page_id_arr:
        idstr = str(id)
        if idstr.find('riben/') != -1 and idstr.find('.html') != -1: #筛选页码地址
            page_id_arr_enable.append(idstr)
    print("页面所有pageId", page_id_arr_enable)
    return page_id_arr_enable

# 获取某页PAGEID中的所有相册链接地址
def find_all_album_of_page(page_url):
    print("当前页面地址", page_url)
    pattern = r'href="([^"]*/a/[0-9]{3,5}/)"[^>]*>.+?' # 匹配href
    html = url_open(page_url).decode('utf-8')
    alllinks = re.findall(pattern,html)
    print("页面所有相册链接地址：" , alllinks)
    return alllinks

# 进入某个相册获取所有图片链接
def get_down_all_pic_of_album(folder, album_url):
    pattern = r'src="([^"]*/[0-9]{3,5}/[0-9]{1,2}.jpg)"[^>]*>.+?' # 匹配href
    html = url_open(album_url).decode('utf-8')
    img_addrs = re.findall(pattern, html)
    #https://img.yituyu.com/gallery/1070/00_qUYWQyHT.jpg
    img_addrs_enable = [] # 过滤不需要的链接
    for h in img_addrs : 
        x = str(h)
        if x.find("https://") != -1 and x.find(".com") != -1 and x.find(".jpg") != -1  :
            img_addrs_enable.append(x)
    print(album_url + "页面所有图片下载地址：" , img_addrs_enable)
    # 下载此页面的所有图片
    album_name = str(album_url)
    save_imgs(folder,img_addrs_enable, album_name) 
    
# 保存图片到文件夹
def save_imgs(folder, img_addrs, album_name):
    sub_dir = album_name.replace("/","").replace(":","").replace(".","")
    os.chdir(folder)
    dirname = sub_dir
    makeDir(dirname)
    os.chdir(dirname)
    for i in img_addrs:
        pic_down_link = str(i)
#         file_name_str = pic_down_link.split('/')
#         filename = file_name_str[-1] # 文件名称 
#         ranInt = ''.join(random.sample(string.ascii_letters + string.digits, 8))# 使用随机数
        filename = pic_down_link.replace("/","").replace(":","")
        pic_pre = "" # 图片下载前缀
        real_down_url = pic_pre + pic_down_link
        if(album_name.split("/")[-2] == real_down_url.split("/")[-2]):
#             print("图片下载地址", real_down_url)
            image = url_open(real_down_url)
#             print("文件名：" + filename)
            with open(filename,'wb') as f:
                f.write(image)
                f.close()
                print("图片下载完成", real_down_url)

# 创建文件夹
def makeDir(folder):
    if os.path.exists(folder) :
        print(folder + "文件夹已存在")
        pass
    try:
        os.mkdir(folder)
    except OSError:
        pass

def download_some(folder='ooxx',pages=10):
    current_dir = "D:\GitHubProject\jupyter-notebook\AllMyTest\\downLoad\\"
    folder = current_dir + folder
    print("文件存储文件夹：",folder)
    makeDir(folder) #新建文件夹
    os.chdir(folder) #跳转到文件夹
    folder_top = os.getcwd() #获取当前工作目录
#     baseurl='https://www.a8e0e744fa9b81a7.com/piclist.x'
#     url = 'https://www.a8e0e744fa9b81a7.com/piclist.x?classid=6'
#     url = 'https://www.tujigu.com/s/35/'
    url = 'https://www.tujigu.com/riben/'
    page_id_arr = get_page_id_arr(url) #获取网页中的pageId
    for page_id in page_id_arr:
        page_url_pre = "https://www.tujigu.com" # page前缀
        page_url = page_url_pre + page_id #组合page访问地址
        all_album_link_arr = find_all_album_of_page(page_url) #获取PageId相册地址
        # 进入页面找到播放地址
        for album_link in all_album_link_arr:
            # 图片服务器可能和访问服务器不是一个,需要组合
            album_pre = ""
            real_album_link = album_pre + album_link
            # 进入页面下载此页面所有图片
            get_down_all_pic_of_album(folder, real_album_link)
            

if __name__ == '__main__':
    folder = input("下载到哪里？(default is 'ooxx'): " )
    pages  = input("下载多少页？(default is 10): ")
    download_some(str(folder),int(pages))
    

下载到哪里？(default is 'ooxx'): dkajsfdkasf
下载多少页？(default is 10): 2
文件存储文件夹： D:\GitHubProject\jupyter-notebook\AllMyTest\downLoad\dkajsfdkasf
页面所有pageId ['/riben/2.html', '/riben/3.html', '/riben/4.html', '/riben/5.html', '/riben/6.html', '/riben/7.html', '/riben/8.html', '/riben/9.html', '/riben/10.html', '/riben/50.html', '/riben/2.html']
当前页面地址 https://www.tujigu.com/riben/2.html
页面所有相册链接地址： ['https://www.tujigu.com/a/23198/', 'https://www.tujigu.com/a/23198/', 'https://www.tujigu.com/a/24684/', 'https://www.tujigu.com/a/24684/', 'https://www.tujigu.com/a/8032/', 'https://www.tujigu.com/a/8032/', 'https://www.tujigu.com/a/19675/', 'https://www.tujigu.com/a/19675/', 'https://www.tujigu.com/a/38186/', 'https://www.tujigu.com/a/38186/', 'https://www.tujigu.com/a/38185/', 'https://www.tujigu.com/a/38185/', 'https://www.tujigu.com/a/38184/', 'https://www.tujigu.com/a/38184/', 'https://www.tujigu.com/a/38183/', 'https://www.tujigu.com/a/38183/', 'https://www.tujigu.com/a/38182/', 'https://www

In [22]:
import re

html = '<a href="/tag/qiaotun/index_2.html">2</a>'
pattern = r'href="([^"]*)"[^>]*>.+?' # 匹配href
print(re.findall(pattern,html))
pattern2 = r'src="([^"]*)"+?'
html2 ='<a href="/xinggan/21610.html" title="PartyCat轰趴猫 风骚翘臀少妇浴室性感蕾丝内衣摄影图片" target="_blank"><img src="https://xintu.crmvscrm.com/d/file/t/20201130/10/4ai2bpnkhlk.jpg" data-original="https://xintu.crmvscrm.com/d/file/t/20201130/10/4ai2bpnkhlk.jpg" alt="PartyCat轰趴猫 风骚翘臀少妇浴室性感蕾丝内衣摄影图片" style="display: inline;"></a>'
print(re.findall(pattern2,html2))
a = 'https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=3075027813,1435505077&amp;fm=26&amp;gp=0.jpg'
b = a.split('/')
print(b[3])
mm = '/search/index?ct=201326592&amp;'
print(mm[17:30])

['/tag/qiaotun/index_2.html']
['https://xintu.crmvscrm.com/d/file/t/20201130/10/4ai2bpnkhlk.jpg']
70cFvHSh_Q1YnxGkpoWK1HF6hhy
201326592&amp


In [6]:
# html ='<a src="https://tjg.hywly.com/a/1/37903/1.jpg" target="_blank"><img src="https://lns.hywly.com/ad/yituyu/shu/1069.jpg" original="https://lns.hywly.com/ad/yituyu/shu/1069.jpg"></a>'
# pattern = r'src="([^"]*/[0-9]{3,5}/[0-9]{1,2}.jpg)"[^>]*>.+?' # 匹配href # 匹配href
# alllinks = re.findall(pattern,html)
# print(alllinks)
print("https://www.tujigu.com/a/31266/".split("/")[-2])

31266
